In [232]:
import pandas as pd
import os
from datetime import datetime
import numpy as np

pd.set_option('max_columns',60)

In [233]:
AMZN_statement_folder = "../data/timeseries/AMZN/Statement"
file_list = []
for dirname, _, filenames in os.walk(f"{AMZN_statement_folder}"):
    for filename in filenames:
        file_list.append(os.path.join(dirname,filename))

In [234]:
list_num = 2
print(file_list[list_num])
df = pd.read_csv(file_list[list_num],index_col=[0,1])
df

../data/timeseries/AMZN/Statement/Consolidated Statements of Cash Flows.csv


,,2020-03-31,2019-12-31,2019-09-30,2019-06-30,2019-03-31,2018-12-31,2018-09-30,2018-06-30,2018-03-31,2017-12-31,2017-09-30,2017-06-30,2017-03-31,2016-12-31,2016-09-30,2016-06-30,2016-03-31,2015-12-31,2015-09-30,2015-06-30,2015-03-31,2014-12-31,2014-09-30,2014-06-30,2014-03-31,2013-12-31,2013-09-30,2013-06-30,2013-03-31,2012-12-31,2012-09-30,2012-06-30,2012-03-31,2011-12-31,2011-09-30,2010-12-31
label,xbrl_tag,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NaN,us-gaap_cashcashequivalentsrestrictedcashandrestrictedcashequivalents,2.750500e+10,3.641000e+10,2.355400e+10,2.296500e+10,2.350700e+10,3.217300e+10,2.103200e+10,2.053600e+10,1.761600e+10,2.185600e+10,1.396000e+10,1.385100e+10,1.630100e+10,1.993400e+10,1.412400e+10,1.297700e+10,1.278100e+10,1.617500e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Net income,us-gaap_netincomeloss,2.535000e+09,1.158800e+10,2.134000e+09,2.625000e+09,3.561000e+09,1.007300e+10,2.883000e+09,2.534000e+09,1.629000e+09,3.033000e+09,2.560000e+08,1.970000e+08,7.240000e+08,2.371000e+09,2.520000e+08,8.570000e+08,5.130000e+08,5.960000e+08,7.900000e+07,9.200000e+07,-5.700000e+07,-2.410000e+08,-4.370000e+08,-1.260000e+08,1.080000e+08,2.740000e+08,-4.100000e+07,NaN,8.200000e+07,NaN,-2.740000e+08,NaN,NaN,NaN,NaN,NaN
"Depreciation, Depletion and Amortization",us-gaap_depreciationdepletionandamortization,5.362000e+09,2.178900e+10,5.563000e+09,5.202000e+09,4.854000e+09,1.534100e+10,3.778000e+09,3.630000e+09,3.671000e+09,1.147800e+10,2.912000e+09,2.633000e+09,2.435000e+09,8.116000e+09,2.084000e+09,1.909000e+09,1.827000e+09,6.281000e+09,1.599000e+09,1.504000e+09,1.426000e+09,4.746000e+09,1.247000e+09,1.109000e+09,1.010000e+09,3.253000e+09,8.340000e+08,NaN,7.000000e+08,NaN,5.540000e+08,NaN,NaN,NaN,NaN,NaN
"Share-based Payment Arrangement, Noncash Expense",us-gaap_sharebasedcompensation,1.757000e+09,6.864000e+09,1.779000e+09,1.971000e+09,1.274000e+09,5.418000e+09,1.350000e+09,1.468000e+09,1.182000e+09,4.215000e+09,1.085000e+09,1.158000e+09,7.920000e+08,2.975000e+09,7.760000e+08,7.680000e+08,5.440000e+08,2.119000e+09,5.440000e+08,5.630000e+08,4.070000e+08,1.497000e+09,3.770000e+08,3.910000e+08,3.210000e+08,1.134000e+09,2.810000e+08,NaN,2.290000e+08,NaN,2.170000e+08,NaN,NaN,NaN,NaN,NaN
"Other Operating Activities, Cash Flow Statement",us-gaap_otheroperatingactivitiescashflowstatement,6.700000e+07,1.640000e+08,4.700000e+07,8.000000e+07,-1.300000e+07,2.740000e+08,6.200000e+07,8.500000e+07,5.600000e+07,2.020000e+08,4.300000e+07,6.000000e+07,NaN,NaN,3.100000e+07,5.300000e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other Noncash Income (Expense),us-gaap_othernoncashincomeexpense,-5.650000e+08,2.490000e+08,-3.880000e+08,7.000000e+06,1.350000e+08,-2.190000e+08,-9.600000e+07,-1.100000e+08,1.840000e+08,2.920000e+08,1.280000e+08,1.200000e+08,4.000000e+07,2.000000e+07,2.300000e+07,-3.100000e+07,5.200000e+07,-2.450000e+08,-5.600000e+07,-1.800000e+07,-9.100000e+07,-6.200000e+07,-4.200000e+07,8.000000e+06,5.000000e+07,-1.660000e+08,-5.000000e+06,NaN,-6.800000e+07,NaN,-1.570000e+08,NaN,NaN,NaN,NaN,NaN
Deferred Income Tax Expense (Benefit),us-gaap_deferredincometaxexpensebenefit,3.220000e+08,7.960000e+08,9.200000e+07,1.050000e+08,4.150000e+08,4.410000e+08,2.660000e+08,-1.390000e+08,1.410000e+08,-2.900000e+07,-7.400000e+07,3.760000e+08,-2.200000e+07,-2.460000e+08,-8.100000e+07,1.060000e+08,1.100000e+07,8.100000e+07,-6.300000e+07,-4.300000e+07,-2.000000e+06,-3.160000e+08,-2.700000e+08,-4.900000e+07,-1.850000e+08,-1.560000e+08,1.100000e+07,NaN,-8.000000e+07,NaN,-3.600000e+07,NaN,NaN,NaN,NaN,NaN
Increase (Decrease) in Inventories,us-gaap_increasedecreaseininventories,-1.392000e+09,3.278000e+09,3.810000e+08,2.100000e+09,-7.190000e+08,1.314000e+09,1.094000e+09,1.090000e+09,-2.220000e+09,3.583000e+09,1.593000e+09,6.820000e+08,-9.470000e+08,1.426000e+09,1.095000e+09,5.700000e+07,-7.690000e+08,2.187000e+09,1.537000e+09,2.700000e+07,-7.210000e+08,1.193000e+09,8.450000e+08,-9.200000e+07

In [221]:
df.columns

Index(['2020-03-31', '2019-12-31', '2019-09-30', '2019-06-30', '2019-03-31',
       '2018-12-31', '2018-09-30', '2018-06-30', '2018-03-31', '2017-12-31',
       '2017-09-30', '2017-06-30', '2017-03-31', '2016-12-31', '2016-09-30',
       '2016-06-30', '2016-03-31', '2015-12-31', '2015-09-30', '2015-06-30',
       '2015-03-31', '2014-12-31', '2014-09-30', '2014-06-30', '2014-03-31',
       '2013-12-31', '2013-09-30', '2013-06-30', '2013-03-31', '2012-12-31',
       '2012-09-30', '2012-06-30', '2012-03-31', '2011-12-31', '2011-09-30',
       '2011-06-30', '2011-03-31', '2010-12-31', '2010-09-30', '2010-06-30',
       '2010-03-31', '2009-12-31', '2009-09-30', '2009-06-30', '2009-03-31',
       '2008-12-31', '2008-09-30', '2008-06-30', '2007-12-31'],
      dtype='object')

In [224]:
sequential_quarters = 0
for i in range(1,len(df.columns)):
    diff = datetime.strptime(df.columns[i-1],'%Y-%m-%d') - datetime.strptime(df.columns[i],'%Y-%m-%d')
    if diff.days > 85 and diff.days < 95:
        sequential_quarters +=1
    else:
        break
    
    

In [225]:
sequential_quarters

47

In [209]:
expected_columns = round((datetime.strptime(df.columns.max(), "%Y-%m-%d")-datetime.strptime(df.columns.min(), "%Y-%m-%d")).days/(365/4))+1
expected_columns

50

In [210]:
len(df.columns)

49

In [87]:
(datetime.strptime(df.columns.max(), "%Y-%m-%d")-datetime.strptime(df.columns.min(), "%Y-%m-%d")).days/(365/4)



43.035616438356165